In [ ]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

Clone the repository and get the data:



In [1]:
!git clone https://github.com/SwiftPredator/How-Does-Bert-Answer-QA-DLP2021/

!mv "/content/How-Does-Bert-Answer-QA-DLP2021/src/probing-tasks/data" "/content/"
!mv "/content/How-Does-Bert-Answer-QA-DLP2021/src/probing-tasks/replicate" "/content/"


fatal: destination path 'How-Does-Bert-Answer-QA-DLP2021' already exists and is not an empty directory.
mv: cannot stat '/content/How-Does-Bert-Answer-QA-DLP2021/src/probing-tasks/data': No such file or directory
mv: cannot stat '/content/How-Does-Bert-Answer-QA-DLP2021/src/probing-tasks/replicate': No such file or directory


In [ ]:
%cd /content/replicate
!pip install -r requirements.txt
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

/content/replicate
     |████████████████████████████████| 2.5MB 8.0MB/s 
     |████████████████████████████████| 901kB 50.4MB/s 
     |████████████████████████████████| 3.3MB 46.9MB/s 


In [2]:
%cd /content/replicate

from edge_probing_utils import (
    JiantDatasetSingleSpan,
    JiantDatasetTwoSpan
    )

import edge_probing as ep
import edge_probing_tpu as ep_tpu
import torch
import torch.nn as nn
import torch.utils.data as data
from transformers import AutoModel, AutoTokenizer


/content/replicate


**Setup:**

In [3]:
tasks = [
    #"ner", 
    #"semeval",
    "coref",
    #"sup-squad",
    #"ques",
    #"sup-babi",
    #"sup-hotpot",
    ]

task_type = {
    "ner": "single_span", 
    "semeval": "single_span",
    "coref": "two_span",
    "sup-squad": "two_span",
    "ques": "single_span",
    "sup-babi": "two_span",
    "sup-hotpot": "two_span",
    }

models = [
    "bert-base-uncased", 
    "csarron/bert-base-uncased-squad-v1"
    ]

task_label_to_id = {
    "coref": {"0": 0, "1": 1},
    "sup-squad": {"0": 0, "1": 1},
    "sup-babi": {"0": 0, "1": 1},
    }

model_to_linestyle = {
    "bert-base-uncased": ":g", 
    "csarron/bert-base-uncased-squad-v1": "-y",
}

# Pick a dataset size: small, medium, big
size = "medium"

In [ ]:
import torch_xla
import torch_xla.core.xla_model as xm
import os
import json
import matplotlib.pyplot as plt
from google.colab import drive

drive.mount('/content/drive')

loss_function = nn.BCELoss()
batch_size = 32
num_layers = range(1,13,2)
num_workers = 0

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = xm.xla_device()
tpu = True
#device = torch.device("cpu")

# Disable warnings.
os.environ["TOKENIZERS_PARALLELISM"] = "false"

os.makedirs(f"/content/drive/MyDrive/replicated/probing-results/", exist_ok=True)
if os.path.isfile(f'content/drive/MyDrive/replicated/probing-results/{size}.json'):
  with open(f'../results/{size}.json', 'r') as f:
    results = json.load(f)
else:
  results = {}


for model in models:
    tokenizer = AutoTokenizer.from_pretrained(model)
    for task in tasks:
        task_results = results.setdefault(task, {})
        label_to_id = task_label_to_id[task]
        train_data = ep.tokenize_jiant_dataset(
            tokenizer,
            *(ep.read_jiant_dataset(f"../data/{task}/{size}/train.jsonl")),
            label_to_id,
            )
        val_data = ep.tokenize_jiant_dataset(
            tokenizer,
            *(ep.read_jiant_dataset(f"../data/{task}/{size}/val.jsonl")),
            label_to_id,
            )
        if task_type[task] == "single_span":
            train_data = JiantDatasetSingleSpan(train_data)
            val_data = JiantDatasetSingleSpan(val_data)
        elif task_type[task] == "two_span":
            train_data = JiantDatasetTwoSpan(train_data)
            val_data = JiantDatasetTwoSpan(val_data)
        train_loader = data.DataLoader(train_data, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=num_workers)
        val_loader = data.DataLoader(val_data, batch_size=batch_size, pin_memory=True, num_workers=num_workers)
        if tpu:
            results[task][model] = ep_tpu.probing(
                train_loader,
                val_loader,
                val_loader,
                model,
                num_layers,
                loss_function,
                label_to_id,
                task_type[task],
                dev=device,
                epochs=1,
            )
        else:
            results[task][model] = ep.probing(
                train_loader,
                val_loader,
                val_loader,
                model,
                num_layers,
                loss_function,
                label_to_id,
                task_type[task],
                dev=device,
                epochs=1,   
            )

with open(f'content/drive/MyDrive/replicated/probing-results/{size}.json', 'r') as f:
    json.dump(results, f)

100%|██████████| 2647/2647 [00:00<00:00, 76357.10it/s]

Reading ../data/coref/medium/train.jsonl
Tokenizing



100%|██████████| 500/500 [00:00<00:00, 6346.62it/s]


Reading ../data/coref/medium/val.jsonl
Tokenizing
Probing layer 1 of 11


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertEdgeProbingTwoSpan: ['bert.encoder.layer.6.attention.self.query.bias', 'bert.encoder.layer.1.output.LayerNorm.weight', 'bert.encoder.layer.5.attention.output.dense.weight', 'bert.encoder.layer.7.intermediate.dense.weight', 'bert.encoder.layer.4.output.LayerNorm.bias', 'bert.encoder.layer.10.attention.self.key.weight', 'bert.encoder.layer.5.intermediate.dense.bias', 'bert.encoder.layer.9.attention.self.query.bias', 'bert.encoder.layer.7.attention.output.dense.bias', 'bert.encoder.layer.6.attention.self.key.weight', 'bert.encoder.layer.5.output.LayerNorm.weight', 'bert.encoder.layer.6.attention.output.LayerNorm.bias', 'bert.encoder.layer.2.attention.self.query.bias', 'bert.encoder.layer.8.attention.self.key.weight', 'bert.encoder.layer.4.attention.self.query.weight', 'bert.encoder.layer.3.attention.output.dense.bias', 'bert.encoder.layer.5.attention.self.query.bias', 'bert.encoder.layer.5.attent

In [ ]:
with open(f'content/drive/MyDrive/replicated/probing-results/{size}.json', 'r') as f:
    results = json.load(f)
for task, task_results in results.items():
    for model, _ in task_results.items():
        ep.plot_task(results, task, model,model_to_linestyle[model], num_layers)
    plt.show()